In [9]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os

In [10]:
# Define the path to the dataset directory
dataset_path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT"

# Get the list of image files
image_files = []
labels = []

# Iterate over the directories
for class_name in ['benign', 'malignant', 'normal']:
    class_path = os.path.join(dataset_path, class_name)
    for filename in os.listdir(class_path):
        image_path = os.path.join(class_path, filename)
        image_files.append(image_path)
        labels.append(class_name)

In [11]:
# Convert labels to numerical values
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Convert labels to one-hot encoded format
labels = to_categorical(labels)

# Split the data into train, validation, and test sets with 80/10/10
X_train_full, X_test, y_train_full, y_test = train_test_split(image_files, labels, test_size=0.2, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.1111, random_state=42, stratify=y_train_full)


In [12]:
# Define a function to load and preprocess the images
def load_preprocess_image(image_path, target_size=(150, 150)):
    """Loads and preprocesses an image."""
    image = Image.open(image_path)
    image = image.resize(target_size)
    image = image.convert('RGB')  # Convert to RGB
    image = np.array(image)
    image = image / 255.0
    return image

In [13]:

# Load and preprocess the training images and labels
X_train = [load_preprocess_image(image_path) for image_path in X_train]
X_train = np.array(X_train)
y_train = np.array(y_train)

# Load and preprocess the validation images and labels
X_val = [load_preprocess_image(image_path) for image_path in X_val]
X_val = np.array(X_val)
y_val = np.array(y_val)

# Load and preprocess the testing images and labels
X_test = [load_preprocess_image(image_path) for image_path in X_test]
X_test = np.array(X_test)
y_test = np.array(y_test)

# Print the shapes of the resulting datasets
print("Train data shape:", X_train.shape, y_train.shape)
print("Validation data shape:", X_val.shape, y_val.shape)
print("Test data shape:", X_test.shape, y_test.shape)


Train data shape: (1121, 150, 150, 3) (1121, 3)
Validation data shape: (141, 150, 150, 3) (141, 3)
Test data shape: (316, 150, 150, 3) (316, 3)


In [14]:

# Initialize the model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Add more convolutional layers
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train, y_train, batch_size=32, epochs=70, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model on training and test data
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print('Training Accuracy:', train_accuracy, "Train loss:", train_loss)



Epoch 1/70
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 825ms/step - accuracy: 0.8583 - loss: 0.3467 - val_accuracy: 0.7305 - val_loss: 0.9495
Epoch 2/70
36/36 ━━━━━━━━━━━━━━━━━━━━ 29s 811ms/step - accuracy: 0.8704 - loss: 0.2981 - val_accuracy: 0.7092 - val_loss: 1.0108
Epoch 3/70
36/36 ━━━━━━━━━━━━━━━━━━━━ 29s 819ms/step - accuracy: 0.8639 - loss: 0.3207 - val_accuracy: 0.7730 - val_loss: 0.7788
Epoch 4/70
36/36 ━━━━━━━━━━━━━━━━━━━━ 41s 822ms/step - accuracy: 0.9094 - loss: 0.2330 - val_accuracy: 0.7801 - val_loss: 1.0019
Epoch 5/70
36/36 ━━━━━━━━━━━━━━━━━━━━ 41s 819ms/step - accuracy: 0.9397 - loss: 0.1524 - val_accuracy: 0.7589 - val_loss: 1.4014
Epoch 6/70
36/36 ━━━━━━━━━━━━━━━━━━━━ 29s 815ms/step - accuracy: 0.9474 - loss: 0.1441 - val_accuracy: 0.6312 - val_loss: 2.0328
Epoch 7/70
36/36 ━━━━━━━━━━━━━━━━━━━━ 41s 810ms/step - accuracy: 0.8044 - loss: 0.6879 - val_accuracy: 0.7163 - val_loss: 0.7526
Epoch 8/70
36/36 ━━━━━━━━━━━━━━━━━━━━ 42s 830ms/step - accuracy: 0.8900 - loss: 0.3092 - val_accu

In [36]:


# Example: Predicting using an image
def predict_image(image_path):
    preprocessed_image = load_preprocess_image(image_path)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Add an extra dimension for batch size
    detect = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(detect[0])
    classes = ['benign', 'malignant', 'normal']
    predicted_class = classes[predicted_class_index]
    return predicted_class

# Example usage: Replace 'image_path' with the path to your image
image_path = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/normal (88).png'
predicted_class = predict_image(image_path)
print("Predicted Class:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Class: normal


In [39]:
# Save the model
model.save('BreastCancer_TrainModel.h5')

